**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Thu Aug 19 08:19:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15298, done.
remote: Total 15298 (delta 0), reused 0 (delta 0), pack-reused 15298
Receiving objects: 100% (15298/15298), 13.68 MiB | 10.42 MiB/s, done.
Resolving deltas: 100% (10398/10398), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "jerry_can" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-08-16 19:25:15--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  65.8MB/s    in 2.4s    

2021-08-16 19:25:17 (65.8 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/classes.txt    
  inflating: data/obj/__MACOSX/._classes.txt  
  inflating: data/obj/gas1.jpeg      
  inflating: data/obj/__MACOSX/._gas1.jpeg  
  inflating: data/obj/gas1.txt       
  inflating: data/obj/__MACOSX/._gas1.txt  
  inflating: data/obj/gas2.jpeg      
  inflating: data/obj/__MACOSX/._gas2.jpeg  
  inflating: data/obj/gas2.txt       
  inflating: data/obj/__MACOSX/._gas2.txt  
  inflating: data/obj/gas3.jpeg      
  inflating: data/obj/__MACOSX/._gas3.jpeg  
  inflating: data/obj/gas3.txt       
  inflating: data/obj/__MACOSX/._gas3.txt  
  inflating: data/obj/gas4.jpeg      
  inflating: data/obj/__MACOSX/._gas4.jpeg  
  inflating: data/obj/gas4.txt       
  inflating: data/obj/__MACOSX/._gas4.txt  
  inflating: data/obj/gas5.jpeg      
  inflating: data/obj/__MACOSX/._gas5.jpeg  
  inflating: data/obj/gas5.txt       
  inflating: data/obj/gas6.jpeg      
  inflating: data/obj/__MACOSX/._gas6.jpeg  
  inflating: d

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.493333 0.491111 0.462222 0.813333

['15', '0.493333', '0.491111', '0.462222', '0.813333']
0 data/obj/gas116.txt
0 0.493333 0.491111 0.462222 0.813333
15 0.489071 0.552727 0.781421 0.829091

['15', '0.489071', '0.552727', '0.781421', '0.829091']
1 data/obj/gas100.txt
0 0.489071 0.552727 0.781421 0.829091
15 0.486667 0.502222 0.582222 0.826667

['15', '0.486667', '0.502222', '0.582222', '0.826667']
2 data/obj/images (23).txt
0 0.486667 0.502222 0.582222 0.826667
15 0.516949 0.470175 0.661017 0.687719

['15', '0.516949', '0.470175', '0.661017', '0.687719']
3 data/obj/images (25).txt
0 0.516949 0.470175 0.661017 0.687719
15 0.486486 0.490842 0.843243 0.673993

['15', '0.486486', '0.490842', '0.843243', '0.673993']
4 data/obj/gas7.txt
0 0.486486 0.490842 0.843243 0.673993
15 0.500000 0.508197 0.700483 0.836066

['15', '0.500000', '0.508197', '0.700483', '0.836066']
5 data/obj/gas144.txt
0 0.500000 0.508197 0.700483 0.836066
15 0.497778 0.497778 0.506667 0.924444

['15', '0.497778', '0.

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpeg")
print(images_list)

['data/obj/gas126.jpeg', 'data/obj/gas100.jpeg', 'data/obj/images (10).jpeg', 'data/obj/gas146.jpeg', 'data/obj/gas18.jpeg', 'data/obj/gas85.jpeg', 'data/obj/gas136.jpeg', 'data/obj/images (53).jpeg', 'data/obj/gas87.jpeg', 'data/obj/gas43.jpeg', 'data/obj/image12.jpeg', 'data/obj/gas139.jpeg', 'data/obj/gas38.jpeg', 'data/obj/gas27.jpeg', 'data/obj/gas48.jpeg', 'data/obj/gas80.jpeg', 'data/obj/image9.jpeg', 'data/obj/gas40.jpeg', 'data/obj/gas10.jpeg', 'data/obj/gas83.jpeg', 'data/obj/gas26.jpeg', 'data/obj/gas17.jpeg', 'data/obj/images.jpeg', 'data/obj/gas8.jpeg', 'data/obj/gas24.jpeg', 'data/obj/gas55.jpeg', 'data/obj/gas144.jpeg', 'data/obj/gas62.jpeg', 'data/obj/gas137.jpeg', 'data/obj/images (13).jpeg', 'data/obj/image68.jpeg', 'data/obj/images (58).jpeg', 'data/obj/gas127.jpeg', 'data/obj/images (32).jpeg', 'data/obj/images (64).jpeg', 'data/obj/images (43).jpeg', 'data/obj/gas53.jpeg', 'data/obj/images (49).jpeg', 'data/obj/gas37.jpeg', 'data/obj/image63.jpeg', 'data/obj/images

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 139504, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.832454), count: 4, class_loss = 0.000241, iou_loss = 0.116084, total_loss = 0.116325 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 139508, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.911938), count: 5, class_loss = 0.001615, iou_loss = 0.039076, total_loss = 0.040692 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (ms